In [1]:
# default_exp config

# Config

Load a .env in the project folder and use env variable if they exist. Env variable overwrite .env config.

In [2]:
import unittest

tc = unittest.TestCase()

In [3]:
# export
import os

from decouple import AutoConfig, Config, Csv, RepositoryEnv, UndefinedValueError


class ConfigNotFound(Exception):
    pass


class Configuration:
    project_name = ""
    config = None
    env_filepath = None

    def __init__(
        self, project_folder="leximpact-socio-fiscal-simu-etat", env_filepath=None
    ):
        """
        ::project_folder:: The folder of your project, where the .env is.
        Does not work if your .env is in a subfolder of the place you launch Python
        """
        self.project_name = project_folder
        self.env_filepath = env_filepath
        self.get_project_module_path()
        self.load_config()

    def get_project_module_path(self):
        cur_path = os.getcwd()
        root_pos = cur_path.find(self.project_name) + len(self.project_name)
        self.project_module_path = cur_path[:root_pos]
        return self.project_module_path

    def load_config(self):
        try:
            if self.env_filepath:
                self.config = Config(RepositoryEnv(self.env_filepath))
            else:
                self.config = Config(
                    RepositoryEnv(os.path.join(self.project_module_path, ".env"))
                )
        except FileNotFoundError as e:
            print("WARNING : ", e)
            self.config = AutoConfig()
        return self.config

    def __str__(self):
        msg = f"Config loaded from {self.project_module_path}/(.env ou ENV)"
        # for env, value in self.config.items():
        #     msg += f"\n{env}={value}"
        return msg

    def print_config(self, logger):
        logger.info(f"Config loaded from {self.project_module_path}/(.env ou ENV)")
        # for env, value in self.config.items():
        #     logger.info(f"{env}={value}")

    def get(self, param_name, fail_on_missing=True, **kwargs):
        try:
            return self.config(param_name, **kwargs)
        except UndefinedValueError:
            if fail_on_missing:
                raise ConfigNotFound(
                    f"Parameter {param_name} not found in {self.project_module_path + '/(.env ou ENV)'}"
                )
            else:
                return None

    def set(self, param_name, value):
        os.environ[param_name] = str(value)

    def Csv(self, **kwargs):
        return Csv(**kwargs)

### Tests

In [4]:
os.environ["TEST_CONFIG_LEXIMPACT_INT"] = "42"
os.environ["TEST_CONFIG_LEXIMPACT_CSV"] = "oui, non, peut-etre"

In [5]:
conf = Configuration()
print(conf)

Config loaded from /home/jupyter-benoit/leximpact-socio-fiscal-simu-etat/(.env ou ENV)


In [6]:
!echo $TEST_CONFIG_LEXIMPACT_CSV

oui, non, peut-etre


In [7]:
conf.get("TEST_CONFIG_LEXIMPACT_INT", cast=int)

42

In [8]:
tc.assertEqual(conf.get("TEST_CONFIG_LEXIMPACT_INT", cast=int), 42)
tc.assertEqual(
    conf.get("TEST_CONFIG_LEXIMPACT_CSV", cast=conf.Csv()), ["oui", "non", "peut-etre"]
)

In [9]:
#!cat ../.env

In [10]:
conf.get("ERFS")

'/mnt/data-out/leximpact/erfs-fpr/'

In [11]:
conf.get("ERFS")

'/mnt/data-out/leximpact/erfs-fpr/'

In [12]:
tc.assertRaises(ConfigNotFound, conf.get, ("TOTO"))

In [13]:
tc.assertIsNone(conf.get("TOTO", fail_on_missing=False))

In [14]:
conf.set("TOTO", "titi")
tc.assertEqual(conf.get("TOTO"), "titi")

In [15]:
tc.assertTrue(conf.get("NOT_EXIST", default=True, cast=bool))

## Utilisation

Import du module

In [16]:
from leximpact_socio_fisca_simu_etat.config import Configuration

Lecture d'un paramètre

In [17]:
config = Configuration()
config.get("ERFS")

'/mnt/data-out/leximpact/erfs-fpr/'